When you interact with an LLM, naturally, it doesn't remember the previous messages.
We can overcome this by creating `memory`.
`LangChain` offers multiple types of `memory` management techniques.

# `ConversationBufferMemory`

In [1]:
from langchain.chains import ConversationChain
from langchain.chat_models import AzureChatOpenAI
from langchain.memory import ConversationBufferMemory

We initialize our `model` as normal, and initialize a `ConversationChain` with the model and add `memory`.

In [2]:
api_version = "2023-12-01-preview"
deployment_id = "gpt-35-turbo-16k"

In [3]:
chat = AzureChatOpenAI(model=deployment_id, temperature=0.0, api_version=api_version)
memory = ConversationBufferMemory()
convo = ConversationChain(
    memory=memory,
    llm=chat,
)

Note that we use the `predict` method with keyword argument `input`.
This is a result of using a `chain` instead of the `chat` as in the first lesson.

In [4]:
convo.predict(input="Hi, my name is Ian")

"Hello Ian! It's nice to meet you. How can I assist you today?"

In [5]:
convo.predict(input="What is 1+1?")

'1+1 is equal to 2.'

In [6]:
convo.predict(input="What is my name?")

'Your name is Ian.'

Because we are using `memory`, the model can remember our prior messages.
If we set `verbose=True` in the `ConversationChain` we can see more of what is happening under the hood.

In [7]:
memory = ConversationBufferMemory()
convo = ConversationChain(
    memory=memory,
    verbose=True,
    llm=chat,
)

In [8]:
convo.predict(input="Hi, my name is Ian")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is Ian
AI:

> Finished chain.


"Hello Ian! It's nice to meet you. How can I assist you today?"

Everything that is in <span style="color:green">green</span> is a part of the memory and any internal prompts.

In [9]:
convo.predict(input="What is 1+1?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Ian
AI: Hello Ian! It's nice to meet you. How can I assist you today?
Human: What is 1+1?
AI:

> Finished chain.


'1+1 is equal to 2.'

As the conversation grows, you can see the <span style="color:green">Current conversation</span> being updated with prior messages.
The prompt tells the LLM what has already been said, giving it context to answer follow-up questions.

In [10]:
convo.predict(input="What is my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Ian
AI: Hello Ian! It's nice to meet you. How can I assist you today?
Human: What is 1+1?
AI: 1+1 is equal to 2.
Human: What is my name?
AI:

> Finished chain.


'Your name is Ian.'

Hence why it knows what my name is 🙂.

## Side Notes

Here is what it looks like if we exclude `memory`.

In [11]:
chat = AzureChatOpenAI(model=deployment_id, temperature=0.0, api_version=api_version)
convo = ConversationChain(
    verbose=True,
    llm=chat,
)

In [12]:
convo.predict(input="Hi, my name is Ian")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is Ian
AI:

> Finished chain.


"Hello Ian! It's nice to meet you. How can I assist you today?"

In [13]:
convo.predict(input="What is 1+1?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Ian
AI: Hello Ian! It's nice to meet you. How can I assist you today?
Human: What is 1+1?
AI:

> Finished chain.


'1+1 is equal to 2.'

In [14]:
convo.predict(input="What is my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Ian
AI: Hello Ian! It's nice to meet you. How can I assist you today?
Human: What is 1+1?
AI: 1+1 is equal to 2.
Human: What is my name?
AI:

> Finished chain.


'Your name is Ian.'

It actually looks like the `memory` argument didn't need to be set 🤔.

In [15]:
ConversationChain?

Init signature:
ConversationChain(
    *,
    memory: langchain_core.memory.BaseMemory = None,
    callbacks: Union[List[langchain_core.callbacks.base.BaseCallbackHandler], langchain_core.callbacks.base.BaseCallbackManager, NoneType] = None,
    callback_manager: Optional[langchain_core.callbacks.base.BaseCallbackManager] = None,
    verbose: bool = None,
    tags: Optional[List[str]] = None,
    metadata: Optional[Dict[str, Any]] = None,
    prompt: langchain_core.prompts.base.BasePromptTemplate = PromptTemplate(input_variables=['history', 'input'], template='The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.\n\nCurrent conversation:\n{history}\nHuman: {input}\nAI:'),
    llm: Union[langchain_core.runnables.base.Runnable[Union[langchain_core.prompt_values.PromptValue, str, List[langchain_core.messages.base.B

The docs say that the `memory` argument should subclass from `langchain_core.memory.BaseMemory` but defaults to `None.

In [16]:
from inspect import getsource

In [17]:
print(getsource(ConversationChain))

class ConversationChain(LLMChain):
    """Chain to have a conversation and load context from memory.

    Example:
        .. code-block:: python

            from langchain.chains import ConversationChain
            from langchain.llms import OpenAI

            conversation = ConversationChain(llm=OpenAI())
    """

    memory: BaseMemory = Field(default_factory=ConversationBufferMemory)
    """Default memory store."""
    prompt: BasePromptTemplate = PROMPT
    """Default conversation prompt to use."""

    input_key: str = "input"  #: :meta private:
    output_key: str = "response"  #: :meta private:

    class Config:
        """Configuration for this pydantic object."""

        extra = Extra.forbid
        arbitrary_types_allowed = True

    @classmethod
    def is_lc_serializable(cls) -> bool:
        return False

    @property
    def input_keys(self) -> List[str]:
        """Use this since so some prompt vars come from history."""
        return [self.input_key]

    @root_

Looking at the source code, `memory` is set to:

```python
memory: BaseMemory = Field(default_factory=ConversationBufferMemory)
```

This means that if we don't provide an argument for `memory`, a `ConversationBufferMemory` instance is used by default.
Here is an example function.

In [18]:
from collections import Counter

from pydantic.fields import Field
from pydantic.main import BaseModel

class Example(BaseModel):
    """This is an example class.
    
    It highlights how the `default_factory` argument works in `Field`.
    """

    counter: dict = Field(default_factory=Counter)


eg = Example()
print(eg.counter)  # Counter()

Counter()


## Continuing

If we look at the `memory` instance we can view what has been added to its `buffer`.

In [35]:
memory = ConversationBufferMemory()
convo = ConversationChain(
    memory=memory,
    llm=chat,
)
convo.predict(input="Hi, my name is Ian")
convo.predict(input="What is 1+1?")
convo.predict(input="What is my name?")

print(memory.buffer)

Human: Hi, my name is Ian
AI: Hello Ian! It's nice to meet you. How can I assist you today?
Human: What is 1+1?
AI: 1+1 is equal to 2.
Human: What is my name?
AI: Your name is Ian.


We can also view the memory as a dictionary of variables, with `history` holding the prior messages in string format.

In [36]:
memory.load_memory_variables({})

{'history': "Human: Hi, my name is Ian\nAI: Hello Ian! It's nice to meet you. How can I assist you today?\nHuman: What is 1+1?\nAI: 1+1 is equal to 2.\nHuman: What is my name?\nAI: Your name is Ian."}

The `memory` doesn't have to be modified by an LLM -- we can update it ourselves.

In [37]:
memory = ConversationBufferMemory()
memory.save_context(inputs={"input": "Hi"}, outputs={"output": "What's up"})

print(memory.buffer)

Human: Hi
AI: What's up


In [38]:
memory.load_memory_variables({})

{'history': "Human: Hi\nAI: What's up"}

As we add to more inputs and outputs to the context, the buffer is updated.

In [39]:
memory.save_context(inputs={"input": "Not much, just hanging"}, outputs={"output": "Cool"})

memory.load_memory_variables({})

{'history': "Human: Hi\nAI: What's up\nHuman: Not much, just hanging\nAI: Cool"}

# `ConversationBufferWindowMemory`